In [1]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior () 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
st = tf.SparseTensor(values=[1, 2], indices=[[0, 0], [1, 1]], dense_shape=[2, 2])
dt = tf.ones(shape=[2,2],dtype=tf.int32)
result = tf.sparse_tensor_dense_matmul(st,dt)
sess = tf.Session()
with sess.as_default():
    print(result.eval())


[[1 1]
 [2 2]]


In [11]:
tf.nn.softmax([1.0])

<tf.Tensor 'Softmax_4:0' shape=(1,) dtype=float32>

In [14]:
import numpy as np

In [15]:
#TRIGRAM_D  表示letter trigrim 之后的维度
TRIGRAM_D = 1000

#定义query输入和doc输入
query_batch = tf.sparse_placeholder(tf.float32, 
                                    shape=[None,TRIGRAM_D], 
                                    name='QueryBatch')
doc_batch = tf.sparse_placeholder(tf.float32, 
                                    shape=[None, TRIGRAM_D], 
                                    name='DocBatch')

In [16]:
#第一层输出维度
L1_N = 300
l1_par_range = np.sqrt(6.0 / (TRIGRAM_D + L1_N))
weight1 = tf.Variable(tf.random_uniform([TRIGRAM_D, L1_N], 
                                        -l1_par_range, 
                                        l1_par_range))
bias1 = tf.Variable(tf.random_uniform([L1_N], 
                                       -l1_par_range, 
                                       l1_par_range))

#因为数据比较稀疏，所以用sparse_tensor_dense_matmul
query_l1 = tf.sparse_tensor_dense_matmul(query_batch, weight1) + bias1
doc_l1 = tf.sparse_tensor_dense_matmul(doc_batch, weight1) + bias1

#激活层，也可以换成别的激活函数
query_l1_out = tf.nn.tanh(query_l1)
doc_l1_out = tf.nn.tanh(doc_l1)

In [18]:
query_l1_out

<tf.Tensor 'Tanh:0' shape=(?, 300) dtype=float32>

In [21]:
#第二层的输出维度
L2_N = 300
l2_par_range = np.sqrt(6.0 / (L1_N+ L2_N))
weight2 = tf.Variable(tf.random_uniform([L1_N, L2_N], 
                                        -l2_par_range, 
                                        l2_par_range))
bias2 = tf.Variable(tf.random_uniform([L2_N], 
                                       -l2_par_range, 
                                       l2_par_range))

query_l2 = tf.matmul(query_l1_out , weight2) + bias2
doc_l2 = tf.matmul(doc_l1_out , weight2) + bias2

query_l2_out = tf.nn.tanh(query_l2)
doc_l2_out = tf.nn.tanh(doc_l2)

#第三层
L3_N = 128
l3_par_range = np.sqrt(6.0 / (L2_N+ L3_N))
weight3 = tf.Variable(tf.random_uniform([L2_N, L3_N], 
                                        -l3_par_range, 
                                        l3_par_range))
bias3 = tf.Variable(tf.random_uniform([L3_N], 
                                       -l3_par_range, 
                                       l3_par_range))

query_l3 = tf.matmul(query_l2_out , weight3) + bias3
doc_l3 = tf.matmul(doc_l2_out , weight3) + bias3

query_l3_out = tf.nn.tanh(query_l3)
doc_l3_out = tf.nn.tanh(doc_l3)

In [28]:
#NEG表示负样本的个数
NEG  = 4

# ||yq||
query_norm = tf.tile(tf.sqrt(tf.reduce_sum(tf.square(query_l3_out ), 1, True)), 
                     [NEG + 1, 1])
# ||yd||
doc_norm = tf.sqrt(tf.reduce_sum(tf.square(doc_l3_out), 1, True))

# yqT yd
prod = tf.reduce_sum(tf.multiply(tf.tile(query_l3_out , [NEG + 1, 1]), doc_l3_out), 1, True)
norm_prod = tf.multiply(query_norm, doc_norm)

# cosine
cos_sim_raw = tf.truediv(prod, norm_prod)
#cos_sim = tf.transpose(tf.reshape(tf.transpose(cos_sim_raw), [NEG + 1, BS])) * Gamma
cos_sim = cos_sim_raw

In [29]:
cos_sim_raw

<tf.Tensor 'truediv_2:0' shape=(?, 1) dtype=float32>

In [31]:
#BS为batch_size，计算batch平均损失

prob = tf.nn.softmax((cos_sim))

#正例的softmax值
hit_prob = tf.slice(prob, [0, 0], [-1, 1])

#最小化loss，计算batch的平均损失
loss = -tf.reduce_sum(tf.log(hit_prob)) / 1000

In [34]:
loss,prob,hit_prob

(<tf.Tensor 'truediv_3:0' shape=() dtype=float32>,
 <tf.Tensor 'Softmax_6:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Slice_1:0' shape=(?, 1) dtype=float32>)

In [35]:
#定义优化方法和学习率
train_step = tf.train.GradientDescentOptimizer(FLAGS.learning_rate).minimize(loss)

with tf.Session(config=config) as sess:
    sess.run(tf.initialize_all_variables())
    for step in range(FLAGS.max_steps):
        sess.run(train_step, feed_dict={query_batch : ...
                                        doc_batch   : ...}})

SyntaxError: invalid syntax (<ipython-input-35-0a97e00b935b>, line 8)